In [1]:
# Step 1: Import Libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.applications.resnet import ResNet50# Correct ResNet50 import
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Correct ImageDataGenerator import
import matplotlib.pyplot as plt
import numpy as np


# Step 2: Define Paths and Preprocessing
image_size = (224, 224)
batch_size = 32

# Load datasets
train_ds = tf.keras.utils.image_dataset_from_directory(
    "metal_nut/train",
    labels="inferred",
    label_mode="binary",  # Binary classification: good (0) or defective (1)
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "metal_nut/test",
    labels="inferred",
    label_mode="binary",
    image_size=image_size,
    batch_size=batch_size
)

# Normalize the pixel values
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Split test data for validation (optional)
val_ds = test_ds.take(20)  # Adjust size as needed
test_ds = test_ds.skip(20)

# Step 3: Build the Model (Transfer Learning)
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

# Add classification layers
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")  # Binary classification
])

# Compile the model
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Step 4: Train the Model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,  # Adjust based on performance
)

# Step 5: Evaluate the Model
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc}")

# Step 6: Visualize Training Performance
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Step 7: Save the Model
model.save("defect_detection_model.h5")
print("Model saved successfully!")

# Step 8: Use the Model for Predictions
def predict_image(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=image_size)
    img_array = image.img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    return "Defective" if prediction[0] > 0.5 else "Good"

# Test the prediction function
img_path = "metal_nut/test/good/000.png"  # Change to your test image path
result = predict_image(img_path)
print(f"The predicted result is: {result}")


Found 220 files belonging to 1 classes.


ValueError: When passing `label_mode="binary"`, there must be exactly 2 class_names. Received: class_names=['good']